In [1]:
import numpy as np
np.set_printoptions(threshold = np.inf)
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
from matplotlib.pyplot import figure
np.random.seed(seed=1)
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import Contours, Histogram2dContour, Marker, Scatter

init_notebook_mode(connected=True)


In [0]:
def gf2elim(M):
    m,n = M.shape
    i=0
    j=0
    while i < m and j < n:
        # find value and index of largest element in remainder of column j
        k = np.argmax(M[i:, j]) +i

        # swap rows
        #M[[k, i]] = M[[i, k]] this doesn't work with numba
        temp = np.copy(M[k])
        M[k] = M[i]
        M[i] = temp


        aijn = M[i, j:]

        col = np.copy(M[:, j]) #make a copy otherwise M will be directly affected

        col[i] = 0 #avoid xoring pivot row with itself

        flip = np.outer(col, aijn)

        M[:, j:] = M[:, j:] ^ flip

        i += 1
        j +=1
    res = []
    for i in range (m):
        res.append(M[i][n - 1])
        print(i, M[i][n-1])
    return res

### SET Global Params

In [0]:
# число слоёв в матрице
l = 3
# число единиц в строке
n0 = 4
# число строк в слое
m = 5

# число столбцов и одновременно длина кода
n = n0 * m
# число строк и проверок
r = l * m

In [0]:
## Create H
H = np.zeros((r, n)).astype(int)
for i in range(m):
    for j in range(n0):
        H[i][i * n0 + j] = 1

for i in range(1, l):
    temp = H[:][0:m].T.copy()
    np.random.shuffle(temp)
    H[:][i * m : (i+1) * m] = temp.T.copy()

# print(H)
# print(H.shape)

In [0]:
def decode(y, H):# x передаваемое слово
    change = 1
    cur_weight = 0
    prev_weight = 0
    prev_prev_weight = 0
    while(change > 0):
    #     проверки для слова
        check = np.zeros(r)

        for i in range(r):
            check[i] = np.sum(y * H[i]) % 2
        cur_weight = int(np.sum(check))
#         print(f'\nВес синдрома - {cur_weight}')

        if (cur_weight == prev_prev_weight):
#             print('Отказ от декодирования')
            break
        prev_prev_weight = prev_weight
        prev_weight = cur_weight

        change = 0
        imax = 0
        _max = 0
        for i in range(n):
            count = int(np.sum(H.T[i] * check))

            if count > _max:
                _max = count
                imax = i

        if _max >= l // 2:
            change += 1
            y[imax] = 1 - y[imax]
    return y
# print(decode(y = np.zeros(n) + np.round(np.random.rand(n) < alpha).astype(int),H=H))

In [0]:
H_copy = H.copy()
swap = []
for i in range(r):
    if not H_copy[r - i - 1][n - i - 1]:
        flag = False
        for j in range(i+1, r):
            if H_copy[r - j - 1][n - i - 1]:
                H_copy[[r - i - 1, r - j - 1]] = H_copy[[r - j - 1, r - i - 1]]
                flag = True
                break
        if not flag:
            for j in range(n - i - 1, n):
                if H_copy[r - i - 1][n - j - 1]:
                    H_copy[:, [n - i - 1, n - j - 1]] = H_copy[:, [n - j - 1, n - i - 1]]
                    swap.append([n - i - 1, n - j - 1])
    
    for j in range(i + 1, r):
        if H_copy[r - j - 1][n - i - 1]:
            H_copy[r - j - 1] = H_copy[r - j - 1] ^ H_copy[r - i - 1]
            
del_list = []
for i in range(r):
    if np.sum(H_copy[i]) == 0:
        del_list.append(i)
H_copy = np.delete(H_copy, (del_list), axis = 0)
r -= len(del_list)

for i in range(r):
    if H_copy[r - i - 1, n - i - 1] == 0:
        for j in range(i + 1, n):
            if H_copy[r - i - 1, n - j - 1] == 1:
                H_copy[:, [n - i - 1, n - j - 1]] = H_copy[:, [n - j - 1, n - i - 1]]
                swap.append([n - i - 1, n - j - 1])
                break
for i in range(r):
    if H_copy[i][n - r + i]:
        for j in range(i + 1, r):
            if H_copy[j][n - r + i]:
                H_copy[j] = H_copy[i] ^ H_copy[j]
                

In [0]:
r = H_copy.shape[0]
P = H_copy[:, :n - r].copy().T
# print(P, P.shape)
G = np.append(np.eye(P.shape[0]), P, axis = 1)
##total matrix
H_final = H.copy()
for i in range(len(swap)):
    H_final[:, [swap[i][0], swap[i][1]]] = H_final[:, [swap[i][1], swap[i][0]]]
H = H_final
# print(H_final.shape)

In [0]:
r = l * m

In [9]:
##draw plots 
numOfRuns = int(2e3)
probIn = []
probOut = []
for p in tqdm_notebook(np.linspace(0,0.5,100)):
    fails = 0
    for run in range(numOfRuns):
        x = np.round(np.random.rand(G.shape[0]) < 0.5).astype(int)  
        y = x.dot(G) % 2
        
        y_copy = y.copy()
        y += np.round(np.random.rand(n) < p).astype(int)
        y %= 2
        if not np.array_equal(y_copy, decode(y,H)):
            fails += 1
    probIn.append(p)
    probOut.append(fails / numOfRuns)

In [0]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [11]:
configure_plotly_browser_state()
trace = go.Scatter(
    x = probIn,
    y = probOut
)
layout = go.Layout(
    xaxis=dict(
        autorange=True,
        showgrid=True,
        title='Symbol error probability',
    ),
    yaxis=dict(
        autorange=True,
        showgrid=True,
        title='Total error probability'
    )
   
)
data = [trace]
fig = go.Figure(data=data, layout=layout)
iplot(fig)